In [ ]:
import os # file management
import sys

!{sys.executable} -m pip install nltk
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.


True

Import libraries:

In [ ]:
## install for notebook
!{sys.executable} -m pip install numpy
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install matplotlib
!{sys.executable} -m pip install seaborn
!{sys.executable} -m pip install librosa
!{sys.executable} -m pip install forcealign

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import librosa
from forcealign import ForceAlign

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.9/63.9 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.3/180.3 kB 8.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 58.4 MB/s eta 0:00:00
  Created wheel for sox: filename=sox-1.5.0-py3-none-any.whl size=40036 sha256=cd02114f984882ffb970238c935d7fce546fc84a79f45c76fded354dc329fb02
  Stored in directory: /root/.cache/pip/wheels/8c/c7/e7/baea1f7e79b9eb53addc81cc9b827424f4a7d8c9cc18c03659
  Created wheel for distance: filename=Distance-0.1.3-py3-none-any.whl size=16256 sha256=734dd6670047056ed35a6d1aaf540399577390a4db0754a5c905a7a1969e7a07
  Stored in directory: /root/.cache/pip/wheels/24/a8/58/407063d8e5c1d4dd6594c99d12baa0108570b56a92325587dd
Successfully built sox distance


[nltk_data] Downloading package cmudict to /root/nltk_data...
[nltk_data]   Unzipping corpora/cmudict.zip.
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:300: SyntaxWarning: invalid escape sequence '\('
  m = re.match('([su]([0-9]{1,2})p?) \(([0-9]{1,2}) bit\)$', token)
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:301: SyntaxWarning: invalid escape sequence '\('
  m2 = re.match('([su]([0-9]{1,2})p?)( \(default\))?$', token)
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:310: SyntaxWarning: invalid escape sequence '\('
  elif re.match('(flt)p?( \(default\))?$', token):
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:314: SyntaxWarning: invalid escape sequence '\('
  elif re.match('(dbl)p?( \(default\))?$', token):


Variables:

In [ ]:
file_info = {
    # "BL": ["FU"],
    # "BS": ["Pre", "FU", "Post"],
    # "BW": ["Pre", "FU", "Post"],
    # "DB": ["Pre", "FU", "Post"],
    # "DP": ["Pre", "FU", "Post"],
    # "FO": ["Pre", "FU", "Post"],
    # "MW": ["Pre", "FU", "Post"],
    # "NT": ["Pre", "FU", "Post"],
    # "PM": ["Pre", "FU", "Post"],
    "PO": ["Pre", "Post"],
    # "RL": ["Pre", "FU", "Post"],
    # "TC": ["Pre", "FU", "Post"]
}

ForceAlign: word functions

In [ ]:
"""
return word labels, provided transcript and the file path of the audio file.
"""
def forcealign_label_words(transcript, audio_file_path, is_denoised=True):
    align = ForceAlign(audio_file=audio_file_path+('audio_denoised.wav' if is_denoised else 'audio.wav'), transcript=transcript)

    # Run prediction and return alignment results
    words = align.inference()
    # print(words)
    return words

"""
print word labels to the screen, provided an array of words.
"""
def forcealign_print_words(words):
    # Show predicted word-level alignments
    for word in words:
        print(f"{word.time_start:.3f}\t{word.time_end:.3f}\t{word.word}")

"""
print word labels to the tool-name folder in the outputs folder, provided an array of words.
"""
def forcealign_output_words(words, file_path, is_denoised):
    # full_path = "./outputs/"+file_path+("/denoised/" if is_denoised else "/default/")+"forcealign/"
    full_path = "./outputs/"+file_path
    if not os.path.exists(full_path):
        os.makedirs(full_path)
    with open(full_path+"labels_word.txt", "w") as f:
        for word in words:
            print(f"{word.time_start:.3f}\t{word.time_end:.3f}\t{word.word}", file=f)

In [ ]:
def calculate_error(measured, true):
    return abs(measured - true)

def forcealign_get_errors(folder, number, main_path, file_path, is_denoised, words, output_errors):
	label_words = []
	automated_times = []
	manual_times = []
	errors = []

	for word in words:
		automated_times.append([word.time_start, word.time_end])
		label_words.append(word.word)

	for line in open(main_path+file_path+"labels_word.txt", "r"):
		tokens = line.split()
		manual_times.append([float(tokens[0]), float(tokens[1])])

	# print(main_path, file_path)

	if len(automated_times) != len(manual_times):
		print(f"Manual and automated time labels for {folder}/{number} contains errors.")
		return

	# full_path = "./outputs/"+file_path+("/denoised/" if is_denoised else "/default/")+"forcealign/"
	full_path = "./outputs/"+file_path
	if not os.path.exists(full_path):
		os.makedirs(full_path)
	with open(full_path+"errors_word.txt", "w") as f:
		for i in range(len(automated_times)):
			a_start, a_end = automated_times[i][0], automated_times[i][1]
			m_start, m_end = manual_times[i][0], manual_times[i][1]

			start_error = calculate_error(a_start, m_start)
			end_error = calculate_error(a_end, m_end)

			errors.append([start_error, end_error, label_words[i]])

			if output_errors:
				print(f"{errors[i][0]:.5f}\t{errors[i][1]:.5f}\t{errors[i][2]}", file=f)

	return errors

To CSV:

In [ ]:
import math

def forcealign_csv_output_words(input_main_path, file_path, is_denoised, words):
    audio_file = input_main_path + file_path + ('audio_denoised.wav' if is_denoised else 'audio.wav')
    y, fs = librosa.load(audio_file)

    output_main_path = "./outputs/"

    avm_audacity = pd.read_table(
        # output_main_path + file_path + ('denoised/' if is_denoised else 'default/') + '/forcealign/labels_word.txt',
        output_main_path + file_path + 'labels_word.txt',
        names=['Start','End','Words']
    )

    f0, voiced_flag, voiced_probs = librosa.pyin(y,sr=fs, frame_length=2048, win_length=256,
                                                 fmin=librosa.note_to_hz('C2'),
                                                 fmax=librosa.note_to_hz('C7')
    )
    times = librosa.times_like(f0)
    avmF0librosa = pd.DataFrame()
    avmF0librosa['Time'] = times
    avmF0librosa['Data'] = f0

    # calcaulate the mean F0 for each note in the Audacity annotations
    num_words = len(words)
    noteF0vals = []
    noteF0mean = []

    # with open(output_main_path + file_path + ('denoised/' if is_denoised else 'default/') + '/forcealign/output.csv', "w") as f:
    with open(output_main_path + file_path + '/output.csv', "w") as f:
        # headers = ["ONSET TIME", "AVG PITCH IN HZ", "DURATION"]
        # print(f"{headers[0]}, {headers[1]}, {headers[2]}", file=f)

        for i in range(num_words):
            idx1 = (np.abs(avmF0librosa['Time'] - avm_audacity['Start'][i])).argmin()
            idx2 = (np.abs(avmF0librosa['Time'] - avm_audacity['End'][i])).argmin()
            noteF0vals.append(avmF0librosa['Data'][idx1:idx2])
            avg_pitch_in_hz = np.nanmean(noteF0vals[i])
            noteF0mean.append(avg_pitch_in_hz)

            output_string = f"{words[i].time_start:.3f}, {avg_pitch_in_hz:.3f}, {words[i].time_end - words[i].time_start:.3f}, {words[i].word}"
            if math.isnan(avg_pitch_in_hz):
                # CHANGED nan to 1, to prevent breakage in pyAMPACT
                output_string = f"{words[i].time_start:.3f}, {1}, {words[i].time_end - words[i].time_start:.3f}, {words[i].word}" # CHANGED nan to 1, to prevent breakage in pyAMPACT

            print(output_string, file=f)
    # print(noteF0mean)

Main function:

In [ ]:
def forcealign_generate_data(is_denoised, ignore_ends, print_words, print_errors, output_words, output_errors):
    words_dict = {}
    errors_dict = {}
    for folder, numbers in file_info.items():
        words_dict[folder] = {}
        errors_dict[folder] = {}
        for number in numbers:
            # if folder != "DL" or number != 1:
            #     break

            '''directories and data'''
            main_path = "./data/"
            file_path = folder+"/"+str(number)+"/"

            transcript_file = open(main_path+file_path+"text.txt", "r")
            transcript = transcript_file.readline()


            '''word/phoneme processing functions'''
            # forcealign_label_phonemes(transcript, main_path+file_path, is_denoised)
            words = forcealign_label_words(transcript, main_path+file_path, is_denoised)
            words_dict[folder][number] = words

            if print_words:
                forcealign_print_words(words)

            if output_words:
                forcealign_output_words(words, file_path, is_denoised)


            # ''' error functions '''
            # errors = forcealign_get_errors(folder, number, main_path, file_path, is_denoised, words, output_errors)

            # if errors is None:
            #     print(f"{folder}/{number}: an error occurred when trying to access the errors")
            #     continue

            # num_rows = len(errors)
            # start_accum = 0
            # end_accum = 0
            # for i in range(0, num_rows):
            #     start_accum += 0 if ignore_ends and i == 0 else errors[i][0]
            #     end_accum += 0 if ignore_ends and i == num_rows - 1 else errors[i][1]

            # if ignore_ends and num_rows == 1:
            #     print(f"{folder}/{number}: an error occurred when trying to calculate the errors: text contains only 1 word")
            #     continue
            # start_avg = start_accum/(num_rows if not ignore_ends else num_rows - 1)
            # end_avg = end_accum/(num_rows if not ignore_ends else num_rows - 1)

            # error_string = f"{start_avg:.9}\t{end_avg:.9}\t{folder}/{number}"
            # if print_errors:
            #     print(error_string)
            # errors_dict[folder][number] = {start_avg: start_avg, end_avg: end_avg}


            '''generate csv output'''
            forcealign_csv_output_words(main_path, file_path, is_denoised, words)

    return words_dict, errors_dict


Use main function:

In [ ]:
is_denoised = False # if True: uses the denoised audio file to get the words
ignore_ends = True # if True: ignore the FIRST ONSET time and the LAST OFFSET time
print_words = False
print_errors = False
output_words = True
output_errors = True

words, errors = forcealign_generate_data(is_denoised, ignore_ends, print_words, print_errors, output_words, output_errors)

# print(words)
# print(errors)

/usr/local/lib/python3.12/dist-packages/torchaudio/_backend/utils.py:213: UserWarning: In 2.9, this function's implementation will be changed to use torchaudio.load_with_torchcodec` under the hood. Some parameters like ``normalize``, ``format``, ``buffer_size``, and ``backend`` will be ignored. We recommend that you port your code to rely directly on TorchCodec's decoder instead: https://docs.pytorch.org/torchcodec/stable/generated/torchcodec.decoders.AudioDecoder.html#torchcodec.decoders.AudioDecoder.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchaudio/_backend/ffmpeg.py:88: UserWarning: torio.io._streaming_media_decoder.StreamingMediaDecoder has been deprecated. This deprecation is part of a large refactoring effort to transition TorchAudio into a maintenance phase. The decoding and encoding capabilities of PyTorch for both audio and video are being consolidated into TorchCodec. Please see https://github.com/pytorch/audio/issues/3902 for more information. It will be r